In [48]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import os

In [42]:
source = requests.get('https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports/')

soup = bs(source.text,'html.parser')
page = soup.prettify()

In [69]:
web_fileset = set()
web_set = set()
for i in soup.find_all('a', class_='js-navigation-open'):
    if 'csv' in i['href']:
        web_set.add(i['href'])
        web_fileset.add(i['href'].split('/')[-1])

dir_set = set(os.listdir('./dataset'))
new_data = web_fileset.difference(dir_set)
for i in new_data:
    path = 'https://raw.githubusercontent.com'+''.join(i.split('/blob'))
    # raw_source = requests.get(path)
    # raw_soup = bs(raw_source.text, 'html.parser')
    # csv_path = 'https://raw.githubusercontent.com'+raw_soup.find(id='raw-url')['href']
    print(path)
    df = pd.read_csv(path)
    filename = i.split('/')[-1]
    #print(filename)
    df.to_csv('./dataset/'+filename)


In [66]:
len(os.listdir('./dataset/'))

92